In [2]:
import pandas as pd
import numpy as np
import os
path = 'C:/Users/dubrangala/OneDrive - VMware, Inc/Case Studies/hackerarth_prediction/dataset'
os.chdir(path)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
## Read Master Data
df = pd.read_csv("mymaster_new_v3.csv")

In [4]:
## Read Master Data
#df = pd.read_pickle('./processed_if_hbos_pca_cblof_vae_lof_bags1.pkl')
pred_genetic_disorder = pd.read_csv("pred_genetic_disorder_phase1_new_data.csv")

In [6]:
df.myflag.value_counts()

train    22083
test      9465
Name: myflag, dtype: int64

In [7]:
df[df.myflag=='train'].genetic_disorder_v1.value_counts()

Mitochondrial genetic inheritance disorders     11174
Single-gene inheritance diseases                 8371
Multifactorial genetic inheritance disorders     2260
missing                                           278
Name: genetic_disorder_v1, dtype: int64

In [8]:
df = df[df.genetic_disorder_v1!='missing']
df.myflag.value_counts()

train    21805
test      9465
Name: myflag, dtype: int64

In [9]:
df[df.myflag=='train'].disorder_subclass.value_counts()

Leigh syndrome                         5160
Mitochondrial myopathy                 4405
Cystic fibrosis                        3448
Tay-Sachs                              2833
Diabetes                               1817
Hemochromatosis                        1355
Leber's hereditary optic neuropathy     648
Alzheimer's                             152
Cancer                                   97
Name: disorder_subclass, dtype: int64

In [10]:
df_train = df[df.myflag=='train'].copy()
print("train",df_train.shape)
df_valid = df[df.myflag!='train'].copy()
print("valid",df_valid.shape)
df_train = df_train[~df_train.disorder_subclass.isnull()].copy()
print("train after",df_train.shape)

train (21805, 254)
valid (9465, 254)
train after (19915, 254)


In [11]:
pred_genetic_disorder[['patient_id','lgbm_class']].head()

,patient_id,lgbm_class
0,PID0x4175,Multifactorial genetic inheritance disorders
1,PID0x21f5,Single-gene inheritance diseases
2,PID0x49b8,Mitochondrial genetic inheritance disorders
3,PID0x2d97,Mitochondrial genetic inheritance disorders
4,PID0x58da,Single-gene inheritance diseases


In [12]:
pred_genetic_disorder.pred_multi.value_counts()

Mitochondrial genetic inheritance disorders     4920
Multifactorial genetic inheritance disorders    2367
Single-gene inheritance diseases                2178
Name: pred_multi, dtype: int64

In [13]:
df_valid = df_valid.merge(pred_genetic_disorder[['patient_id','lgbm_class']], on='patient_id', how='inner')
df_valid.shape

(9465, 255)

In [14]:
df_valid.lgbm_class.value_counts()

Mitochondrial genetic inheritance disorders     5102
Single-gene inheritance diseases                2772
Multifactorial genetic inheritance disorders    1591
Name: lgbm_class, dtype: int64

### Model coding - Functions

In [15]:
#import libraries
import lightgbm as lgbm
import pandas as pd
import numpy as np
import os
import datetime as dt
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import MinMaxScaler
randSeed=17

In [16]:
df_train[df_train.myflag=='train'].disorder_subclass.unique()

array(["Leber's hereditary optic neuropathy", 'Cystic fibrosis',
       'Diabetes', 'Leigh syndrome', 'Cancer', 'Tay-Sachs',
       'Hemochromatosis', 'Mitochondrial myopathy', "Alzheimer's"],
      dtype=object)

In [17]:
# Code for converting target string lables to integer and vice versa
class_encoding = {"Leber's hereditary optic neuropathy":0, 
                  'Cystic fibrosis':1,
                  'Diabetes':2, 
                  'Leigh syndrome':3, 
                  'Cancer':4, 
                  'Tay-Sachs':5,
                  'Hemochromatosis':6,
                  'Mitochondrial myopathy':7, 
                  "Alzheimer's":8}
class_decoding = {0:"Leber's hereditary optic neuropathy", 
                  1:'Cystic fibrosis',
                  2:'Diabetes', 
                  3:'Leigh syndrome', 
                  4:'Cancer', 
                  5:'Tay-Sachs',
                  6:'Hemochromatosis',
                  7:'Mitochondrial myopathy', 
                  8:"Alzheimer's"}

In [18]:
df_train['target'] = df_train['disorder_subclass'].apply(lambda x:class_encoding[x])
df_valid['target'] = -1
df_train.target.value_counts()

3    5160
7    4405
1    3448
5    2833
2    1817
6    1355
0     648
8     152
4      97
Name: target, dtype: int64

In [19]:
## Create Dummy from phase 1 predictors
df_train['genetic_disorder_1'] = np.where(df_train['genetic_disorder_v1']=='Mitochondrial genetic inheritance disorders', True,False)
df_train['genetic_disorder_2'] = np.where(df_train['genetic_disorder_v1']=='Single-gene inheritance diseases', True,False)
df_train['genetic_disorder_3'] = np.where(df_train['genetic_disorder_v1']=='Multifactorial genetic inheritance disorders', True,False)

df_valid['genetic_disorder_1'] = np.where(df_valid['lgbm_class']=='Mitochondrial genetic inheritance disorders', True,False)
df_valid['genetic_disorder_2'] = np.where(df_valid['lgbm_class']=='Single-gene inheritance diseases', True,False)
df_valid['genetic_disorder_3'] = np.where(df_valid['lgbm_class']=='Multifactorial genetic inheritance disorders', True,False)


In [20]:
df_valid.head()

,patient_id,patient_age,mothers_age,fathers_age,no._of_previous_abortion,white_blood_cell_count_thousand_per_microliter,inherited_from_father,maternal_gene,respiratory_rate_breathsmin,heart_rate_ratesmin,birth_asphyxia,autopsy_shows_birth_defect_if_applicable,folic_acid_details_periconceptional,ho_serious_maternal_illness,ho_radiation_exposure_xray,ho_substance_abuse,assisted_conception_ivfart,history_of_anomalies_in_previous_pregnancies,birth_defects,blood_test_result,blood_cell_count_mcl,test_1,test_2,test_3,test_4,test_5,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,mcap_institute_name,mcap_respiratory_rate_breathsmin,mcap_heart_rate_ratesmin,mcap_followup,mcap_gender,mcap_birth_asphyxia,mcap_autopsy_shows_birth_defect_if_applicable,mcap_place_of_birth,mcap_folic_acid_details_periconceptional,mcap_ho_serious_maternal_illness,flag_ho_serious_maternal_folic_acid,mcap_ho_radiation_exposure_xray,mcap_ho_substance_abuse,flag_ho_substance_abuse_radiation,mcap_assisted_conception_ivfart,mcap_history_of_anomalies_in_previous_pregnancies,flag_history_of_anomalies_ivfart,mcap_birth_defects,mcap_blood_test_result,mcap_symptom_1to_5,genes_in_mothers_side,paternal_gene,status,parental_consent,followup,gender,place_of_birth,myflag,test_1to5,symptom_1to_5,genetic_disorder,disorder_subclass,genetic_disorder_old,disorder_subclass_v1,genetic_disorder_v1,kbin5_c0,kbin5_c1,kbin5_c2,kbin5_c3,kbin5_c4,kbin5_c5,kbin5_c6,kbin5_c7,kbin5_c8,kbin5_c9,kbin5_c10,kbin5_c11,kbin5_c12,kbin5_c13,kbin5_c14,kbin5_c15,kbin5_c16,kbin5_c17,kbin5_c18,kbin5_c19,kbin5_c20,kbin5_c21,kbin5_c22,kbin5_c23,kbin5_c24,kbin5_c25,kbin5_c26,kbin5_c27,kbin5_c28,kbin5_c29,kbin5_c30,kbin5_c31,kbin5_c32,kbin5_c33,kbin5_c34,kbin5_c35,kbin5_c36,kbin5_c37,kbin5_c38,kbin5_c39,kbin5_c40,kbin5_c41,kbin5_c42,kbin5_c43,kbin5_c44,kbin5_c45,kbin5_c46,kbin5_c47,kbin5_c48,inherited_from_father_count,maternal_gene_count,respiratory_rate_breathsmin_count,heart_rate_ratesmin_count,birth_asphyxia_count,autopsy_shows_birth_defect_if_applicable_count,folic_acid_details_periconceptional_count,ho_serious_maternal_illness_count,ho_radiation_exposure_xray_count,ho_substance_abuse_count,assisted_conception_ivfart_count,history_of_anomalies_in_previous_pregnancies_count,birth_defects_count,blood_test_result_count,genes_in_mothers_side_count,paternal_gene_count,status_count,parental_consent_count,followup_count,gender_count,place_of_birth_count,test_1to5_count,symptom_1to_5_count,dummy_inherited_from_father_no,dummy_inherited_from_father_yes,dummy_maternal_gene_no,dummy_maternal_gene_yes,dummy_respiratory_rate_breathsmin_normal (30_60),dummy_respiratory_rate_breathsmin_tachypnea,dummy_heart_rate_ratesmin_normal,dummy_heart_rate_ratesmin_tachycardia,dummy_birth_asphyxia_no,dummy_birth_asphyxia_not available,dummy_birth_asphyxia_yes,dummy_autopsy_shows_birth_defect_if_applicable_no,dummy_autopsy_shows_birth_defect_if_applicable_not applicable,dummy_autopsy_shows_birth_defect_if_applicable_yes,dummy_folic_acid_details_periconceptional_no,dummy_folic_acid_details_periconceptional_yes,dummy_ho_serious_maternal_illness_no,dummy_ho_serious_maternal_illness_yes,dummy_ho_radiation_exposure_xray_no,dummy_ho_radiation_exposure_xray_not applicable,dummy_ho_radiation_exposure_xray_yes,dummy_ho_substance_abuse_no,dummy_ho_substance_abuse_not applicable,dummy_ho_substance_abuse_yes,dummy_assisted_conception_ivfart_no,dummy_assisted_conception_ivfart_yes,dummy_history_of_anomalies_in_previous_pregnancies_no,dummy_history_of_anomalies_in_previous_pregnancies_yes,dummy_birth_defects_multiple,dummy_birth_defects_singular,dummy_blood_test_result_abnormal,dummy_blood_test_result_inconclusive,dummy_blood_test_result_normal,dummy_blood_test_result_slightly abnormal,dummy_genes_in_mothers_side_no,dummy_genes_in_mothers_side_yes,dummy_paternal_gene_no,dummy_paternal_gene_yes,dummy_status_alive,dummy_status_deceased,dummy_parental_consent_no,dummy_parental_consent_yes,dummy_followup_high,dummy_followup_low,dummy_followup_mi

In [21]:
df_train['target'].value_counts()

3    5160
7    4405
1    3448
5    2833
2    1817
6    1355
0     648
8     152
4      97
Name: target, dtype: int64

In [22]:
def parameterSpace(selected_classfier='xgboost'):  
    
    # XGBOOST earlier exploration
    if(selected_classfier=='xgboost'):
        space = {
        'n_estimators': hp.quniform('n_estimators', 50, 800, 20),
        'max_depth':  hp.choice('max_depth', np.arange(5, 25, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 2, 30, 1),
        'subsample': hp.quniform('subsample', 0.1, 0.9, 0.005),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.09, 0.9, 0.05),
        #'colsample_bylevel': hp.quniform('colsample_bylevel', 0.1, 1.0, 0.1),
        #'colsample_bynode': hp.quniform('colsample_bynode', 0.1, 1.0, 0.1),
        'learning_rate': hp.quniform('learning_rate', 0.015, 0.15, 0.005),
        #'gamma':  hp.choice('gamma', np.arange(0, 3, dtype=int)),
        #'eval_metric': 'auc',
        'early_stopping_rounds':50,
        #'num_parallel_tree':50, # no need
        #'nthread': 900,
        #'updater':'grow_gpu_hist',
        #'n_jobs': 10, # gpu pred doesnot need n_jobs
        #'silent': 1,
        #'predictor':'gpu_predictor',
        #'tree_method': 'gpu_hist',
        #'verbosity':0,
        #'missing':np.nan,
        #'sampling_method':'gradient_based', # works only with gpu_hist
        'seed': randSeed
         }
    elif(selected_classfier=='lightgbm'):
        
        space = {
                       
            'num_leaves': hp.quniform('num_leaves', 20, 800, 10),
            'colsample_bytree': hp.quniform('colsample_bytree', 0.1, 0.95, 0.015),
            'learning_rate':hp.quniform('learning_rate', 0.015, 0.22, 0.005),
            'min_child_samples': hp.quniform('min_child_samples', 2, 100,2),
            'lambda_l1': hp.lognormal('lambda_l1', 1e-8, 3.0),
            'lambda_l2':  hp.lognormal('lambda_l2', 1e-8, 3.0),
            'bagging_freq': hp.quniform('bagging_freq', 2, 50,1),
            'subsample': hp.quniform('subsample', 0.1, 0.95, 0.015),
            'device': 'cpu',
            'first_metric_only':False,
            'n_jobs':-1,
            'eval_names': 'hackathon',
            'tree_learner': 'serial',
            'random_state': randSeed} 
        # 600 runs

    elif(selected_classfier=='RF'):
        space= { 
            'n_estimators': hp.quniform('n_estimators', 50, 1500, 20), #number of trees, change it to 1000 for better results
            'criterion':hp.choice('criterion', ['gini','entropy']),
            'max_depth':  hp.choice('max_depth', np.arange(2, 20, dtype=int)),
            'min_samples_split': hp.quniform('min_samples_split', 2, 200,2),
            'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 5,1),
            'min_weight_fraction_leaf': hp.quniform('min_weight_fraction_leaf', 0.0, 0.48,0.01),
            'max_features':hp.choice('max_features',['auto', 'sqrt', 'log2']),
            'min_impurity_decrease': hp.quniform('min_impurity_decrease', 0.0, 0.9,0.01),
            'oob_score':hp.choice('oob_score',[False, True]),
            'max_samples': hp.quniform('max_samples', 0.01, 0.99,0.01)  
          
        }
       
    else:
        print('Only xgboost, lightgbm and randomforest are supported: RF,lightgbm,xgboost')
        
        
    return space

In [23]:
# merge into common dataframe for computing anomalies
df_train['flag_train'] = 1
df_valid['flag_train'] = 0
df = df_train.append(df_valid)
df.reset_index(drop=True, inplace=True)
del df_train
del df_valid
import gc
gc.collect()

93

#### Pre processing of data

In [24]:
def summary_stat (df):
    missing_cnt = df.isnull().sum()
    missing_prop = round(100*df.isnull().sum()/len(df),2)
    uniqv = df.apply(lambda x: x.nunique())
    ctype = df.apply(lambda x: x.dtype)
    mydf = pd.DataFrame({'Column Type':ctype, 'Number of Unique':uniqv, 'Total missing':missing_cnt, 'missing percentage':missing_prop})
    return(mydf)

mydata_dictinary = summary_stat(df)
mydata_dictinary = mydata_dictinary.reset_index()
mydata_dictinary.head()

,index,Column Type,Number of Unique,Total missing,missing percentage
0,patient_id,object,29380,0,0.0
1,patient_age,float64,1258,0,0.0
2,mothers_age,float64,5230,0,0.0
3,fathers_age,float64,5164,0,0.0
4,no._of_previous_abortion,float64,3936,0,0.0


In [25]:
exclude_list = ['genetic_disorder','disorder_subclass','genetic_disorder_old','disorder_subclass_v1','genetic_disorder_v1','myflag','patient_id']
mydata_dictinary = mydata_dictinary[~mydata_dictinary['index'].isin(exclude_list)]
cat_var = mydata_dictinary[mydata_dictinary['Column Type']=='object']['index']
num_var = mydata_dictinary[mydata_dictinary['Column Type']!='object']['index']

In [26]:
df['target'] = df['target'].fillna(-1)
df = df.drop(cat_var,axis=1)
df.head()

,patient_id,patient_age,mothers_age,fathers_age,no._of_previous_abortion,white_blood_cell_count_thousand_per_microliter,blood_cell_count_mcl,test_1,test_2,test_3,test_4,test_5,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,mcap_institute_name,mcap_respiratory_rate_breathsmin,mcap_heart_rate_ratesmin,mcap_followup,mcap_gender,mcap_birth_asphyxia,mcap_autopsy_shows_birth_defect_if_applicable,mcap_place_of_birth,mcap_folic_acid_details_periconceptional,mcap_ho_serious_maternal_illness,flag_ho_serious_maternal_folic_acid,mcap_ho_radiation_exposure_xray,mcap_ho_substance_abuse,flag_ho_substance_abuse_radiation,mcap_assisted_conception_ivfart,mcap_history_of_anomalies_in_previous_pregnancies,flag_history_of_anomalies_ivfart,mcap_birth_defects,mcap_blood_test_result,mcap_symptom_1to_5,myflag,genetic_disorder,disorder_subclass,genetic_disorder_old,disorder_subclass_v1,genetic_disorder_v1,kbin5_c0,kbin5_c1,kbin5_c2,kbin5_c3,kbin5_c4,kbin5_c5,kbin5_c6,kbin5_c7,kbin5_c8,kbin5_c9,kbin5_c10,kbin5_c11,kbin5_c12,kbin5_c13,kbin5_c14,kbin5_c15,kbin5_c16,kbin5_c17,kbin5_c18,kbin5_c19,kbin5_c20,kbin5_c21,kbin5_c22,kbin5_c23,kbin5_c24,kbin5_c25,kbin5_c26,kbin5_c27,kbin5_c28,kbin5_c29,kbin5_c30,kbin5_c31,kbin5_c32,kbin5_c33,kbin5_c34,kbin5_c35,kbin5_c36,kbin5_c37,kbin5_c38,kbin5_c39,kbin5_c40,kbin5_c41,kbin5_c42,kbin5_c43,kbin5_c44,kbin5_c45,kbin5_c46,kbin5_c47,kbin5_c48,inherited_from_father_count,maternal_gene_count,respiratory_rate_breathsmin_count,heart_rate_ratesmin_count,birth_asphyxia_count,autopsy_shows_birth_defect_if_applicable_count,folic_acid_details_periconceptional_count,ho_serious_maternal_illness_count,ho_radiation_exposure_xray_count,ho_substance_abuse_count,assisted_conception_ivfart_count,history_of_anomalies_in_previous_pregnancies_count,birth_defects_count,blood_test_result_count,genes_in_mothers_side_count,paternal_gene_count,status_count,parental_consent_count,followup_count,gender_count,place_of_birth_count,test_1to5_count,symptom_1to_5_count,dummy_inherited_from_father_no,dummy_inherited_from_father_yes,dummy_maternal_gene_no,dummy_maternal_gene_yes,dummy_respiratory_rate_breathsmin_normal (30_60),dummy_respiratory_rate_breathsmin_tachypnea,dummy_heart_rate_ratesmin_normal,dummy_heart_rate_ratesmin_tachycardia,dummy_birth_asphyxia_no,dummy_birth_asphyxia_not available,dummy_birth_asphyxia_yes,dummy_autopsy_shows_birth_defect_if_applicable_no,dummy_autopsy_shows_birth_defect_if_applicable_not applicable,dummy_autopsy_shows_birth_defect_if_applicable_yes,dummy_folic_acid_details_periconceptional_no,dummy_folic_acid_details_periconceptional_yes,dummy_ho_serious_maternal_illness_no,dummy_ho_serious_maternal_illness_yes,dummy_ho_radiation_exposure_xray_no,dummy_ho_radiation_exposure_xray_not applicable,dummy_ho_radiation_exposure_xray_yes,dummy_ho_substance_abuse_no,dummy_ho_substance_abuse_not applicable,dummy_ho_substance_abuse_yes,dummy_assisted_conception_ivfart_no,dummy_assisted_conception_ivfart_yes,dummy_history_of_anomalies_in_previous_pregnancies_no,dummy_history_of_anomalies_in_previous_pregnancies_yes,dummy_birth_defects_multiple,dummy_birth_defects_singular,dummy_blood_test_result_abnormal,dummy_blood_test_result_inconclusive,dummy_blood_test_result_normal,dummy_blood_test_result_slightly abnormal,dummy_genes_in_mothers_side_no,dummy_genes_in_mothers_side_yes,dummy_paternal_gene_no,dummy_paternal_gene_yes,dummy_status_alive,dummy_status_deceased,dummy_parental_consent_no,dummy_parental_consent_yes,dummy_followup_high,dummy_followup_low,dummy_followup_missing,dummy_gender_ambiguous,dummy_gender_female,dummy_gender_male,dummy_gender_missing,dummy_place_of_birth_home,dummy_place_of_birth_institute,dummy_place_of_birth_not available,dummy_test_1to5_0.0_0.0_0.0_1.0_0.0,dummy_test_1to5_0.0_0.0_0.0_1.0_9.0,dummy_test_1to5_0.0_0.0_0.0_9.0_0.0,dummy_test_1to5_0.0_0.0_0.0_9.0_9.0,dummy_test_1to5_0.0_0.0_9.0_1.0_0.0,dummy_test_1to5_0.0_0.0_9.0_1.0_9.0,dummy_test_1to5_0.0_0.0_9.0_9.0_0.0,dummy_test_1to5_0.0_0.0_9.0_9.0_9.0,d

In [27]:
corr_df = df[df.myflag=='train'][num_var].corr()
upper = corr_df.where(np.triu(np.ones(corr_df.shape), k=1).astype(np.bool))
corNms = [column for column in upper.columns if any(upper[column].abs() > 0.999)]

In [35]:
corNms

['followup_count',
 'place_of_birth_count',
 'dummy_inherited_from_father_no',
 'dummy_inherited_from_father_yes',
 'dummy_maternal_gene_no',
 'dummy_maternal_gene_yes',
 'dummy_respiratory_rate_breathsmin_normal (30_60)',
 'dummy_respiratory_rate_breathsmin_tachypnea',
 'dummy_heart_rate_ratesmin_normal',
 'dummy_heart_rate_ratesmin_tachycardia',
 'dummy_birth_asphyxia_not available',
 'dummy_autopsy_shows_birth_defect_if_applicable_not applicable',
 'dummy_folic_acid_details_periconceptional_no',
 'dummy_folic_acid_details_periconceptional_yes',
 'dummy_ho_serious_maternal_illness_no',
 'dummy_ho_serious_maternal_illness_yes',
 'dummy_ho_radiation_exposure_xray_yes',
 'dummy_assisted_conception_ivfart_no',
 'dummy_assisted_conception_ivfart_yes',
 'dummy_history_of_anomalies_in_previous_pregnancies_no',
 'dummy_history_of_anomalies_in_previous_pregnancies_yes',
 'dummy_birth_defects_multiple',
 'dummy_birth_defects_singular',
 'dummy_genes_in_mothers_side_no',
 'dummy_genes_in_mother

In [28]:
df_names = num_var
df_names

1                                         patient_age
2                                         mothers_age
3                                         fathers_age
4                            no._of_previous_abortion
5      white_blood_cell_count_thousand_per_microliter
                            ...                      
254                                            target
255                                genetic_disorder_1
256                                genetic_disorder_2
257                                genetic_disorder_3
258                                        flag_train
Name: index, Length: 229, dtype: object

In [30]:
drop_list = ['myflag','genetic_disorder','disorder_subclass','genetic_disorder_old','disorder_subclass_v1','genetic_disorder_v1']
df = df.drop(drop_list, axis=1)
df = df.drop(corNms, axis=1) ## highly correlated variables 
# seperating train and valid after anomaly based features
df_valid = df[df.flag_train==0]
df_train = df[df.flag_train==1]
df_valid= df_valid.drop(['flag_train'],axis=1)
df_train= df_train.drop(['flag_train', 'patient_id'],axis=1)

df_valid.reset_index(drop=True, inplace=True)
df_train.reset_index(drop=True, inplace=True)
df_train = df_train.fillna(-9999)
df_valid = df_valid.fillna(-9999)

In [31]:
df_valid.head()

,patient_id,patient_age,mothers_age,fathers_age,no._of_previous_abortion,white_blood_cell_count_thousand_per_microliter,blood_cell_count_mcl,test_1,test_2,test_3,test_4,test_5,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,mcap_institute_name,mcap_respiratory_rate_breathsmin,mcap_heart_rate_ratesmin,mcap_followup,mcap_gender,mcap_birth_asphyxia,mcap_autopsy_shows_birth_defect_if_applicable,mcap_place_of_birth,mcap_folic_acid_details_periconceptional,mcap_ho_serious_maternal_illness,flag_ho_serious_maternal_folic_acid,mcap_ho_radiation_exposure_xray,mcap_ho_substance_abuse,flag_ho_substance_abuse_radiation,mcap_assisted_conception_ivfart,mcap_history_of_anomalies_in_previous_pregnancies,flag_history_of_anomalies_ivfart,mcap_birth_defects,mcap_blood_test_result,mcap_symptom_1to_5,kbin5_c0,kbin5_c1,kbin5_c2,kbin5_c3,kbin5_c4,kbin5_c5,kbin5_c6,kbin5_c7,kbin5_c8,kbin5_c9,kbin5_c10,kbin5_c11,kbin5_c12,kbin5_c13,kbin5_c14,kbin5_c15,kbin5_c16,kbin5_c17,kbin5_c18,kbin5_c19,kbin5_c20,kbin5_c21,kbin5_c22,kbin5_c23,kbin5_c24,kbin5_c25,kbin5_c26,kbin5_c27,kbin5_c28,kbin5_c29,kbin5_c30,kbin5_c31,kbin5_c32,kbin5_c33,kbin5_c34,kbin5_c35,kbin5_c36,kbin5_c37,kbin5_c38,kbin5_c39,kbin5_c40,kbin5_c41,kbin5_c42,kbin5_c43,kbin5_c44,kbin5_c45,kbin5_c46,kbin5_c47,kbin5_c48,inherited_from_father_count,maternal_gene_count,respiratory_rate_breathsmin_count,heart_rate_ratesmin_count,birth_asphyxia_count,autopsy_shows_birth_defect_if_applicable_count,folic_acid_details_periconceptional_count,ho_serious_maternal_illness_count,ho_radiation_exposure_xray_count,ho_substance_abuse_count,assisted_conception_ivfart_count,history_of_anomalies_in_previous_pregnancies_count,birth_defects_count,blood_test_result_count,genes_in_mothers_side_count,paternal_gene_count,status_count,parental_consent_count,gender_count,test_1to5_count,symptom_1to_5_count,dummy_birth_asphyxia_no,dummy_birth_asphyxia_yes,dummy_autopsy_shows_birth_defect_if_applicable_no,dummy_autopsy_shows_birth_defect_if_applicable_yes,dummy_ho_radiation_exposure_xray_no,dummy_ho_radiation_exposure_xray_not applicable,dummy_ho_substance_abuse_no,dummy_ho_substance_abuse_not applicable,dummy_ho_substance_abuse_yes,dummy_blood_test_result_abnormal,dummy_blood_test_result_inconclusive,dummy_blood_test_result_normal,dummy_blood_test_result_slightly abnormal,dummy_followup_high,dummy_followup_low,dummy_gender_ambiguous,dummy_gender_female,dummy_gender_male,dummy_place_of_birth_home,dummy_place_of_birth_institute,dummy_test_1to5_0.0_0.0_0.0_1.0_0.0,dummy_test_1to5_0.0_0.0_0.0_1.0_9.0,dummy_test_1to5_0.0_0.0_0.0_9.0_0.0,dummy_test_1to5_0.0_0.0_0.0_9.0_9.0,dummy_test_1to5_0.0_0.0_9.0_1.0_0.0,dummy_test_1to5_0.0_0.0_9.0_1.0_9.0,dummy_test_1to5_0.0_0.0_9.0_9.0_0.0,dummy_test_1to5_0.0_0.0_9.0_9.0_9.0,dummy_test_1to5_0.0_9.0_0.0_1.0_0.0,dummy_test_1to5_0.0_9.0_0.0_1.0_9.0,dummy_test_1to5_0.0_9.0_0.0_9.0_0.0,dummy_test_1to5_0.0_9.0_0.0_9.0_9.0,dummy_test_1to5_0.0_9.0_9.0_1.0_0.0,dummy_test_1to5_0.0_9.0_9.0_1.0_9.0,dummy_test_1to5_0.0_9.0_9.0_9.0_0.0,dummy_test_1to5_0.0_9.0_9.0_9.0_9.0,dummy_test_1to5_9.0_0.0_0.0_1.0_0.0,dummy_test_1to5_9.0_0.0_0.0_1.0_9.0,dummy_test_1to5_9.0_0.0_0.0_9.0_0.0,dummy_test_1to5_9.0_0.0_0.0_9.0_9.0,dummy_test_1to5_9.0_0.0_9.0_1.0_0.0,dummy_test_1to5_9.0_0.0_9.0_1.0_9.0,dummy_test_1to5_9.0_0.0_9.0_9.0_0.0,dummy_test_1to5_9.0_0.0_9.0_9.0_9.0,dummy_test_1to5_9.0_9.0_0.0_1.0_0.0,dummy_test_1to5_9.0_9.0_0.0_1.0_9.0,dummy_test_1to5_9.0_9.0_0.0_9.0_0.0,dummy_test_1to5_9.0_9.0_0.0_9.0_9.0,dummy_test_1to5_9.0_9.0_9.0_1.0_0.0,dummy_test_1to5_9.0_9.0_9.0_1.0_9.0,dummy_test_1to5_9.0_9.0_9.0_9.0_0.0,dummy_test_1to5_9.0_9.0_9.0_9.0_9.0,dummy_symptom_1to_5_0.0_0.0_0.0_0.0_0.0,dummy_symptom_1to_5_0.0_0.0_0.0_0.0_1.0,dummy_symptom_1to_5_0.0_0.0_0.0_1.0_0.0,dummy_symptom_1to_5_0.0_0.0_0.0_1.0_1.0,dummy_symptom_1to_5_0.0_0.0_1.0_0.0_0.0,dummy_symptom_1to_5_0.0_0.0_1.0_0.0_1.0,dummy_symptom_1to_5_0.0_0.0_1.0_1.0_0.0,dummy_symptom_1to_5_0.0_0.0_1.0_1.0_1.0,dummy_symptom_1to_5_0.0_1.0_0.0_0.0_0.0,dummy_symptom_

In [32]:
X_score = df_valid.drop(['target','patient_id'],axis=1) # Test data to be submitted, target data is set to -1

In [33]:
import datetime

x = datetime.datetime.now()
print(x)

2021-08-08 13:24:22.486509


In [34]:
# predict the majority class
def majorityPredictor(arr):    
    lst = arr.tolist()
    occ_= max(lst,key=lst.count)
    return occ_

def getPredictors_multi(selected_classfier,params,df_train,X_score,n_splits,seed_): 
    track_accuracy=[]
    y_score_tot = []   
    folds=StratifiedKFold(n_splits=int(n_splits), shuffle=True, random_state=seed_)
    for train_index, test_index in folds.split(df_train,df_train['target']):
        split_trn, split_val = df_train.iloc[train_index], df_train.iloc[test_index]
        y_train = split_trn['target']
        X_train = split_trn.drop(['target'],axis=1)
        y_valid = split_val['target']
        X_valid = split_val.drop(['target'],axis=1)
        accuracy_1,y_pred_score = myClassfier_multi(selected_classfier,params,X_train, y_train,X_valid,y_valid,X_score)
        track_accuracy.append(accuracy_1)
        y_score_tot.append(y_pred_score)
        
    mean_accuracy_=  np.mean(track_accuracy)
    print("Mean Accuracy: "+str(mean_accuracy_))
    
    final=pd.DataFrame(y_score_tot).T
   
    return final,mean_accuracy_

param_score=[]
## Creating a loss function for Hyper parameter tuning and saving  hyper parameter values and the corresponding accuracy
##  for the defined parameters
def score_multi(params):
    print(params)
    y_pred_score_multi,multi_acc_ = getPredictors_multi(selected_classfier,params,
                                                                                 df_train,X_score,n_splits=5,
                                                                                 seed_=randSeed)
    
    pd.DataFrame([selected_classfier,params,multi_acc_,
                  datetime.now().strftime('%Y_%m_%d_%H_%M_%S')]).T.to_csv('hyper_multi.csv',mode='a', header=False)

    return -multi_acc_

In [35]:
from datetime import datetime
datetime.now()

datetime.datetime(2021, 8, 8, 13, 24, 25, 735385)

In [36]:
#import hyperopt as hp
from hyperopt import hp
from hyperopt import fmin, tpe
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings("ignore")
import lightgbm as lgb
import xgboost as xgb
from  lightgbm import LGBMClassifier
import xgboost as XGBClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, recall_score, precision_score, log_loss, roc_auc_score, accuracy_score,roc_curve,precision_recall_curve
from datetime import datetime

In [37]:
def parameterSpace(selected_classfier='xgboost'):  
    
    # XGBOOST earlier exploration
    if(selected_classfier=='xgboost'):
        space = {
        'n_estimators': hp.quniform('n_estimators', 50, 800, 20),
        'max_depth':  hp.choice('max_depth', np.arange(5, 25, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 2, 30, 1),
        'subsample': hp.quniform('subsample', 0.1, 0.9, 0.005),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.09, 0.9, 0.05),
        #'colsample_bylevel': hp.quniform('colsample_bylevel', 0.1, 1.0, 0.1),
        #'colsample_bynode': hp.quniform('colsample_bynode', 0.1, 1.0, 0.1),
        'learning_rate': hp.quniform('learning_rate', 0.015, 0.15, 0.005),
        #'gamma':  hp.choice('gamma', np.arange(0, 3, dtype=int)),
        #'eval_metric': 'auc',
        'early_stopping_rounds':50,
        #'num_parallel_tree':50, # no need
        #'nthread': 900,
        #'updater':'grow_gpu_hist',
        #'n_jobs': 10, # gpu pred doesnot need n_jobs
        #'silent': 1,
        #'predictor':'gpu_predictor',
        #'tree_method': 'gpu_hist',
        #'verbosity':0,
        #'missing':np.nan,
        #'sampling_method':'gradient_based', # works only with gpu_hist
        'seed': randSeed
         }
    elif(selected_classfier=='lightgbm'):
        
        space = {
                       
            'num_leaves': hp.quniform('num_leaves', 20, 800, 10),
            'colsample_bytree': hp.quniform('colsample_bytree', 0.1, 0.95, 0.015),
            'learning_rate':hp.quniform('learning_rate', 0.015, 0.22, 0.005),
            'min_child_samples': hp.quniform('min_child_samples', 2, 100,2),
            'lambda_l1': hp.lognormal('lambda_l1', 1e-8, 3.0),
            'lambda_l2':  hp.lognormal('lambda_l2', 1e-8, 3.0),
            'bagging_freq': hp.quniform('bagging_freq', 2, 50,1),
            'subsample': hp.quniform('subsample', 0.1, 0.95, 0.015),
            'device': 'cpu',
            'first_metric_only':False,
            'n_jobs':-1,
            'eval_names': 'hackathon',
            'tree_learner': 'serial',
            'random_state': randSeed} 
        # 600 runs

    elif(selected_classfier=='RF'):
        space= { 
            'n_estimators': hp.quniform('n_estimators', 50, 1500, 20), #number of trees, change it to 1000 for better results
            'criterion':hp.choice('criterion', ['gini','entropy']),
            'max_depth':  hp.choice('max_depth', np.arange(2, 20, dtype=int)),
            'min_samples_split': hp.quniform('min_samples_split', 2, 200,2),
            'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 5,1),
            'min_weight_fraction_leaf': hp.quniform('min_weight_fraction_leaf', 0.0, 0.48,0.01),
            'max_features':hp.choice('max_features',['auto', 'sqrt', 'log2']),
            'min_impurity_decrease': hp.quniform('min_impurity_decrease', 0.0, 0.9,0.01),
            'oob_score':hp.choice('oob_score',[False, True]),
            'max_samples': hp.quniform('max_samples', 0.01, 0.99,0.01)  
          
        }
       
    else:
        print('Only xgboost, lightgbm and randomforest are supported: RF,lightgbm,xgboost')
        
        
    return space

In [38]:
def myClassfier_multi(selected_classfier,params,X_train, y_train,X_valid,y_valid,X_score): 
      
    X_train = X_train.to_numpy()
    X_valid = X_valid.to_numpy()
    X_score = X_score.to_numpy()
    if(selected_classfier=='xgboost'):
        clf = XGBClassifier(#colsample_bytree =           params[ 'colsample_bytree'], 
                                #colsample_bylevel =      params['colsample_bylevel'],
                                #colsample_bynode =       params['colsample_bynode'],                                                            
                                #gamma =                  params['gamma'],
                                learning_rate =          params['learning_rate'], 
                                max_depth =              int(params[ 'max_depth']), 
                                min_child_weight =       int(params[ 'min_child_weight']), 
                                n_estimators =           int(params[ 'n_estimators']), 
                                #nthread =                int(params[ 'nthread']), 
                                objective =              'multi:softprob', 
                                seed =                   params[ 'seed'], 
                                #silent =                 params[ 'silent'], 
                                #subsample =              params[ 'subsample'], 
                                #verbosity  =             params[ 'verbosity'], 
                                #early_stopping_rounds =  int(params[ 'early_stopping_rounds']), 
                                #predictor =              params[ 'predictor'], 
                                #tree_method =            params[ 'tree_method'],
                                #updater =                params['updater'],
                                #num_parallel_tree=50,
                                #n_jobs      =            -1,
                                #sampling_method =        params['sampling_method'],
                                #missing =                np.nan,
                                #use_label_encoder=False
        )
        #clf.set_params(**params)
        #print(clf)
    #    predictions = gbm_model.predict(dvalid,
    #                                    ntree_limit=gbm_model.best_iteration + 1)
    #    predictions = (predictions>0.5)
        clf.fit(X_train,y_train,verbose=0)
    elif(selected_classfier =='lightgbm'):
          
        clf = LGBMClassifier(objective              = 'multiclass',
                            #categorical_feature      = cat_cols,
                            #n_estimators             = int(params[ 'n_estimators']),
                            num_leaves               = int(params[ 'num_leaves']),
                            #max_bin = 64,
                            colsample_bytree         = params[ 'colsample_bytree'],
                            learning_rate            = params[ 'learning_rate'],
                            #early_stopping_round    =int( params[ 'early_stopping_round']),
                            #max_depth                = 30,
                            min_child_samples        = int(params[ 'min_child_samples']) ,
                            #lambda_l1                = params['lambda_l1'],
                            #lambda_l2                = params['lambda_l2'],
                            subsample                = params['subsample'],
                            bagging_freq             = int(params['bagging_freq']),
                            verbosity                = -1,
                            device                   = params[ 'device'],
                            first_metric_only        = params[ 'first_metric_only'],
                            #eval_metric              = params[ 'eval_metric'],
                            #metric                   = params[ 'metric'],
                            n_jobs                   = int(params[ 'n_jobs']),
                            #eval_names               = params[ 'eval_names'],
                            random_state             = int(params[ 'random_state']),
                            tree_learner            = params[ 'tree_learner']) 
        
        clf.fit(X_train,y_train,verbose=0) 
    elif(selected_classfier =='RF'):
          
        clf = RandomForestClassifier( 
                            n_estimators              = int(params['n_estimators']),
                            criterion                 = params['criterion'],
                            max_depth                 = int(params['max_depth']),
                            min_samples_split         = int(params['min_samples_split']),            
                            min_samples_leaf          = int(params['min_samples_leaf']),            
                            min_weight_fraction_leaf  = params['min_weight_fraction_leaf'],     
                            max_features              = params['max_features'],    
                            min_impurity_decrease     = params['min_impurity_decrease'],     
                            oob_score                 = params['oob_score'],
                            max_samples               = params['max_samples']
                            )

        clf.fit(X_train,y_train)
        
    y_pred= clf.predict(X_valid)
    y_score_pred = clf.predict(X_score)
    accuracy_1=accuracy_score(y_valid,y_pred)
    del clf
    gc.collect()
            
            
    return accuracy_1,y_score_pred

In [47]:
## Code for hyper paramter tuning  using Light GBM as the classifier and 1000 evaluations
selected_classfier='lightgbm'
space = parameterSpace(selected_classfier)
       
    # Use the fmin function from Hyperopt to find the best hyperparameters
best_lightgbm = fmin(score_multi, space, algo=tpe.suggest, 
                # trials=trials, 
                max_evals=500,rstate= np.random.RandomState(randSeed))

{'bagging_freq': 15.0, 'colsample_bytree': 0.765, 'device': 'cpu', 'eval_names': 'hackathon', 'first_metric_only': False, 'lambda_l1': 1.2785178546988023, 'lambda_l2': 24.28210493447922, 'learning_rate': 0.17500000000000002, 'min_child_samples': 44.0, 'n_jobs': -1, 'num_leaves': 340.0, 'random_state': 17, 'subsample': 0.825, 'tree_learner': 'serial'}
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=15, subsample_freq=0 will be ignored. Current value: bagging_freq=15                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=15, subsample_freq=0 will be ignored. Current value: bagging_freq=15                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=15, subsample_freq=0 will b

bagging_freq is set=21, subsample_freq=0 will be ignored. Current value: bagging_freq=21                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=21, subsample_freq=0 will be ignored. Current value: bagging_freq=21                               
Mean Accuracy: 0.7080090384132564                                                                                      
{'bagging_freq': 23.0, 'colsample_bytree': 0.21, 'device': 'cpu', 'eval_names': 'hackathon', 'first_metric_only': False, 'lambda_l1': 142.2037375347965, 'lambda_l2': 43.015645618287195, 'learning_rate': 0.18, 'min_child_samples': 16.0, 'n_jobs': -1, 'num_leaves': 110.0, 'random_state': 17, 'subsample': 0.21, 'tree_learner': 'serial'}
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=23, subsample_freq=0 will be ignored. Curren

bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6                                 
Mean Accuracy: 0.7034898317850866                                                                                      
{'bagging_freq': 49.0, 'colsample_bytree

[LightGBM] [Warning]                                                                                                   
bagging_freq is set=13, subsample_freq=0 will be ignored. Current value: bagging_freq=13                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=13, subsample_freq=0 will be ignored. Current value: bagging_freq=13                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=13, subsample_freq=0 will be ignored. Current value: bagging_freq=13                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=13, subsample_freq=0 will be ignored. Current value: bagging_freq=13                               
[LightGBM] [Warning]                    

[LightGBM] [Warning]                                                                                                   
bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5                                 
[LightGBM] [Warning]                    

...

[LightGBM] [Warning]                                                                                                   
bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5                                 
[LightGBM] [Warning]                    

[LightGBM] [Warning]                                                                                                   
bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14                               
[LightGBM] [Warning]                    

[LightGBM] [Warning]                                                                                                   
bagging_freq is set=13, subsample_freq=0 will be ignored. Current value: bagging_freq=13                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=13, subsample_freq=0 will be ignored. Current value: bagging_freq=13                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=13, subsample_freq=0 will be ignored. Current value: bagging_freq=13                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=13, subsample_freq=0 will be ignored. Current value: bagging_freq=13                               
[LightGBM] [Warning]                    

[LightGBM] [Warning]                                                                                                   
bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9                                 
[LightGBM] [Warning]                    

[LightGBM] [Warning]                                                                                                   
bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6                                 
[LightGBM] [Warning]                    

In [48]:
best_lightgbm

{'bagging_freq': 14.0,
 'colsample_bytree': 0.44999999999999996,
 'lambda_l1': 5.906590321997508e-11,
 'lambda_l2': 0.3649650483769115,
 'learning_rate': 0.045,
 'min_child_samples': 80.0,
 'num_leaves': 20.0,
 'subsample': 0.9149999999999999}

In [39]:
lgbm_params={'bagging_freq': 14.0, 'colsample_bytree': 0.44999999999999996, 'device': 'cpu', 'eval_names': 'tanzu',
             'first_metric_only': False, 'lambda_l1': 5.906590321997508e-11, 'lambda_l2': 0.3649650483769115, 
             'learning_rate': 0.045, 'max_depth': -1, 'min_child_samples': 80.0, 'n_jobs': -1, 'num_leaves': 20.0,
             'objective': 'multiclass', 'random_state': 17, 'subsample': 0.92, 'tree_learner': 'serial'}
selected_classfier='lightgbm'
y_pred_score_multi,mean_accuracy_= getPredictors_multi(selected_classfier,lgbm_params,df_train,X_score,
                                                        n_splits=5,seed_=randSeed)

[LightGBM] [Warning] bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14
[LightGBM] [Warning] bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14
[LightGBM] [Warning] bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14
[LightGBM] [Warning] bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14
[LightGBM] [Warning] bagging_freq is set=14, subsample_freq=0 will be ignored. Current value: bagging_freq=14
Mean Accuracy: 0.7225709264373588


In [40]:
y_score_multi_pred= y_pred_score_multi.astype(int).apply(majorityPredictor,axis=1) 
y_score_multi_pred.value_counts()

3    3633
1    2237
2    1509
7    1398
5     430
6     105
8      82
0      71
dtype: int64

In [43]:
multi_class_df=pd.DataFrame()
multi_class_df['sub_class_pred_lgbm'] = y_score_multi_pred
multi_class_df['patient_id'] = df_valid.patient_id.values
multi_class_df['lgb_sub_class'] = multi_class_df['sub_class_pred_lgbm'].apply(lambda x:class_decoding[x])
multi_class_df.lgb_sub_class.value_counts()

Leigh syndrome                         3633
Cystic fibrosis                        2237
Diabetes                               1509
Mitochondrial myopathy                 1398
Tay-Sachs                               430
Hemochromatosis                         105
Alzheimer's                              82
Leber's hereditary optic neuropathy      71
Name: lgb_sub_class, dtype: int64

In [44]:
multi_class_df.to_csv('genetic_disorder_subclass_new_score_v1.csv',index=False)   # 34.645

#### Drop genetic_disorder variables from both train and score

In [47]:
drop_c = ['genetic_disorder_1','genetic_disorder_2','genetic_disorder_3']
df_train = df_train.drop(drop_c, axis=1)
X_score = X_score.drop(drop_c, axis=1)

In [49]:
## Code for hyper paramter tuning  using Light GBM as the classifier and 1000 evaluations
selected_classfier='lightgbm'
space = parameterSpace(selected_classfier)
       
    # Use the fmin function from Hyperopt to find the best hyperparameters
best_lightgbm1 = fmin(score_multi, space, algo=tpe.suggest, 
                # trials=trials, 
                max_evals=100,rstate= np.random.RandomState(randSeed))

{'bagging_freq': 15.0, 'colsample_bytree': 0.765, 'device': 'cpu', 'eval_names': 'hackathon', 'first_metric_only': False, 'lambda_l1': 1.2785178546988023, 'lambda_l2': 24.28210493447922, 'learning_rate': 0.17500000000000002, 'min_child_samples': 44.0, 'n_jobs': -1, 'num_leaves': 340.0, 'random_state': 17, 'subsample': 0.825, 'tree_learner': 'serial'}
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=15, subsample_freq=0 will be ignored. Current value: bagging_freq=15                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=15, subsample_freq=0 will be ignored. Current value: bagging_freq=15                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=15, subsample_freq=0 will b

bagging_freq is set=38, subsample_freq=0 will be ignored. Current value: bagging_freq=38                               
Mean Accuracy: 0.3718804920913884                                                                                      
{'bagging_freq': 17.0, 'colsample_bytree': 0.255, 'device': 'cpu', 'eval_names': 'hackathon', 'first_metric_only': False, 'lambda_l1': 1.484935638644304, 'lambda_l2': 0.20758511163197, 'learning_rate': 0.17500000000000002, 'min_child_samples': 12.0, 'n_jobs': -1, 'num_leaves': 540.0, 'random_state': 17, 'subsample': 0.87, 'tree_learner': 'serial'}
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=17, subsample_freq=0 will be ignored. Current value: bagging_freq=17                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=17, subsample_freq=0 will be i

bagging_freq is set=21, subsample_freq=0 will be ignored. Current value: bagging_freq=21                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=21, subsample_freq=0 will be ignored. Current value: bagging_freq=21                               
Mean Accuracy: 0.3913632939994979                                                                                      
{'bagging_freq': 23.0, 'colsample_bytree': 0.21, 'device': 'cpu', 'eval_names': 'hackathon', 'first_metric_only': False, 'lambda_l1': 142.2037375347965, 'lambda_l2': 43.015645618287195, 'learning_rate': 0.18, 'min_child_samples': 16.0, 'n_jobs': -1, 'num_leaves': 110.0, 'random_state': 17, 'subsample': 0.21, 'tree_learner': 'serial'}
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=23, subsample_freq=0 will be ignored. Curren

bagging_freq is set=31, subsample_freq=0 will be ignored. Current value: bagging_freq=31                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=31, subsample_freq=0 will be ignored. Current value: bagging_freq=31                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=31, subsample_freq=0 will be ignored. Current value: bagging_freq=31                               
Mean Accuracy: 0.34737634948531254                                                                                     
{'bagging_freq': 21.0, 'colsample_bytree': 0.63, 'device': 'cpu', 'eval_names': 'hackathon', 'first_metric_only': False, 'lambda_l1': 0.014497072794450847, 'lambda_l2': 0.6004778465514929, 'learning_rate': 0.195, 'min_child_samples': 26.0, 'n_jobs': -1, 'num_leaves': 340.0, 'rand

bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6                                 
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6                                 
Mean Accuracy: 0.3770524730102937                                                                                      
{'bagging_freq': 27.0, 'colsample_bytree

...

[LightGBM] [Warning]                                                                                                   
bagging_freq is set=24, subsample_freq=0 will be ignored. Current value: bagging_freq=24                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=24, subsample_freq=0 will be ignored. Current value: bagging_freq=24                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=24, subsample_freq=0 will be ignored. Current value: bagging_freq=24                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=24, subsample_freq=0 will be ignored. Current value: bagging_freq=24                               
[LightGBM] [Warning]                    

[LightGBM] [Warning]                                                                                                   
bagging_freq is set=18, subsample_freq=0 will be ignored. Current value: bagging_freq=18                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=18, subsample_freq=0 will be ignored. Current value: bagging_freq=18                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=18, subsample_freq=0 will be ignored. Current value: bagging_freq=18                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=18, subsample_freq=0 will be ignored. Current value: bagging_freq=18                               
[LightGBM] [Warning]                    

[LightGBM] [Warning]                                                                                                   
bagging_freq is set=20, subsample_freq=0 will be ignored. Current value: bagging_freq=20                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=20, subsample_freq=0 will be ignored. Current value: bagging_freq=20                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=20, subsample_freq=0 will be ignored. Current value: bagging_freq=20                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=20, subsample_freq=0 will be ignored. Current value: bagging_freq=20                               
[LightGBM] [Warning]                    

[LightGBM] [Warning]                                                                                                   
bagging_freq is set=28, subsample_freq=0 will be ignored. Current value: bagging_freq=28                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=28, subsample_freq=0 will be ignored. Current value: bagging_freq=28                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=28, subsample_freq=0 will be ignored. Current value: bagging_freq=28                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=28, subsample_freq=0 will be ignored. Current value: bagging_freq=28                               
[LightGBM] [Warning]                    

[LightGBM] [Warning]                                                                                                   
bagging_freq is set=20, subsample_freq=0 will be ignored. Current value: bagging_freq=20                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=20, subsample_freq=0 will be ignored. Current value: bagging_freq=20                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=20, subsample_freq=0 will be ignored. Current value: bagging_freq=20                               
[LightGBM] [Warning]                                                                                                   
bagging_freq is set=20, subsample_freq=0 will be ignored. Current value: bagging_freq=20                               
[LightGBM] [Warning]                    

In [50]:
best_lightgbm1

{'bagging_freq': 47.0,
 'colsample_bytree': 0.8099999999999999,
 'lambda_l1': 0.10551154518636044,
 'lambda_l2': 1.224873675633758e-05,
 'learning_rate': 0.02,
 'min_child_samples': 90.0,
 'num_leaves': 650.0,
 'subsample': 0.945}

In [51]:
lgbm_params={'bagging_freq': 47.0, 'colsample_bytree': 0.8099999999999999, 'device': 'cpu', 'eval_names': 'tanzu',
             'first_metric_only': False, 'lambda_l1': 0.10551154518636044, 'lambda_l2': 1.224873675633758e-05, 
             'learning_rate': 0.02, 'max_depth': -1, 'min_child_samples': 90.0, 'n_jobs': -1, 'num_leaves': 650.0,
             'objective': 'multiclass', 'random_state': 17, 'subsample': 0.945, 'tree_learner': 'serial'}
selected_classfier='lightgbm'
y_pred_score_multi,mean_accuracy_= getPredictors_multi(selected_classfier,lgbm_params,df_train,X_score,
                                                        n_splits=5,seed_=randSeed)

[LightGBM] [Warning] bagging_freq is set=47, subsample_freq=0 will be ignored. Current value: bagging_freq=47
[LightGBM] [Warning] bagging_freq is set=47, subsample_freq=0 will be ignored. Current value: bagging_freq=47
[LightGBM] [Warning] bagging_freq is set=47, subsample_freq=0 will be ignored. Current value: bagging_freq=47
[LightGBM] [Warning] bagging_freq is set=47, subsample_freq=0 will be ignored. Current value: bagging_freq=47
[LightGBM] [Warning] bagging_freq is set=47, subsample_freq=0 will be ignored. Current value: bagging_freq=47
Mean Accuracy: 0.4020085362791866


In [52]:
y_score_multi_pred= y_pred_score_multi.astype(int).apply(majorityPredictor,axis=1) 
y_score_multi_pred.value_counts()

3    3306
1    2829
7    1256
2    1141
5     465
0     290
6     138
8      40
dtype: int64

In [53]:
multi_class_df1=pd.DataFrame()
multi_class_df1['sub_class_pred_lgbm'] = y_score_multi_pred
multi_class_df1['patient_id'] = df_valid.patient_id.values
multi_class_df1['lgb_sub_class'] = multi_class_df1['sub_class_pred_lgbm'].apply(lambda x:class_decoding[x])
multi_class_df1.lgb_sub_class.value_counts()

Leigh syndrome                         3306
Cystic fibrosis                        2829
Mitochondrial myopathy                 1256
Diabetes                               1141
Tay-Sachs                               465
Leber's hereditary optic neuropathy     290
Hemochromatosis                         138
Alzheimer's                              40
Name: lgb_sub_class, dtype: int64

In [54]:
multi_class_df1.to_csv('genetic_disorder_subclass_new_score_v2.csv',index=False)   # 34.645

##### H2O AutoML

In [59]:
from sklearn.model_selection import train_test_split
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score
from sklearn.ensemble import RandomForestRegressor
from collections import Counter

In [55]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.281-b09, mixed mode)
  Starting server from C:\Users\dubrangala\Anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\DUBRAN~1\AppData\Local\Temp\tmpz27t5e0w
  JVM stdout: C:\Users\DUBRAN~1\AppData\Local\Temp\tmpz27t5e0w\h2o_dubrangala_started_from_python.out
  JVM stderr: C:\Users\DUBRAN~1\AppData\Local\Temp\tmpz27t5e0w\h2o_dubrangala_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.4
H2O_cluster_version_age:,30 days
H2O_cluster_name:,H2O_from_python_dubrangala_kfmorb
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.052 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"accepting new members, healthy"


In [56]:
df_train.head()

,patient_age,mothers_age,fathers_age,no._of_previous_abortion,white_blood_cell_count_thousand_per_microliter,blood_cell_count_mcl,test_1,test_2,test_3,test_4,test_5,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,mcap_institute_name,mcap_respiratory_rate_breathsmin,mcap_heart_rate_ratesmin,mcap_followup,mcap_gender,mcap_birth_asphyxia,mcap_autopsy_shows_birth_defect_if_applicable,mcap_place_of_birth,mcap_folic_acid_details_periconceptional,mcap_ho_serious_maternal_illness,flag_ho_serious_maternal_folic_acid,mcap_ho_radiation_exposure_xray,mcap_ho_substance_abuse,flag_ho_substance_abuse_radiation,mcap_assisted_conception_ivfart,mcap_history_of_anomalies_in_previous_pregnancies,flag_history_of_anomalies_ivfart,mcap_birth_defects,mcap_blood_test_result,mcap_symptom_1to_5,kbin5_c0,kbin5_c1,kbin5_c2,kbin5_c3,kbin5_c4,kbin5_c5,kbin5_c6,kbin5_c7,kbin5_c8,kbin5_c9,kbin5_c10,kbin5_c11,kbin5_c12,kbin5_c13,kbin5_c14,kbin5_c15,kbin5_c16,kbin5_c17,kbin5_c18,kbin5_c19,kbin5_c20,kbin5_c21,kbin5_c22,kbin5_c23,kbin5_c24,kbin5_c25,kbin5_c26,kbin5_c27,kbin5_c28,kbin5_c29,kbin5_c30,kbin5_c31,kbin5_c32,kbin5_c33,kbin5_c34,kbin5_c35,kbin5_c36,kbin5_c37,kbin5_c38,kbin5_c39,kbin5_c40,kbin5_c41,kbin5_c42,kbin5_c43,kbin5_c44,kbin5_c45,kbin5_c46,kbin5_c47,kbin5_c48,inherited_from_father_count,maternal_gene_count,respiratory_rate_breathsmin_count,heart_rate_ratesmin_count,birth_asphyxia_count,autopsy_shows_birth_defect_if_applicable_count,folic_acid_details_periconceptional_count,ho_serious_maternal_illness_count,ho_radiation_exposure_xray_count,ho_substance_abuse_count,assisted_conception_ivfart_count,history_of_anomalies_in_previous_pregnancies_count,birth_defects_count,blood_test_result_count,genes_in_mothers_side_count,paternal_gene_count,status_count,parental_consent_count,gender_count,test_1to5_count,symptom_1to_5_count,dummy_birth_asphyxia_no,dummy_birth_asphyxia_yes,dummy_autopsy_shows_birth_defect_if_applicable_no,dummy_autopsy_shows_birth_defect_if_applicable_yes,dummy_ho_radiation_exposure_xray_no,dummy_ho_radiation_exposure_xray_not applicable,dummy_ho_substance_abuse_no,dummy_ho_substance_abuse_not applicable,dummy_ho_substance_abuse_yes,dummy_blood_test_result_abnormal,dummy_blood_test_result_inconclusive,dummy_blood_test_result_normal,dummy_blood_test_result_slightly abnormal,dummy_followup_high,dummy_followup_low,dummy_gender_ambiguous,dummy_gender_female,dummy_gender_male,dummy_place_of_birth_home,dummy_place_of_birth_institute,dummy_test_1to5_0.0_0.0_0.0_1.0_0.0,dummy_test_1to5_0.0_0.0_0.0_1.0_9.0,dummy_test_1to5_0.0_0.0_0.0_9.0_0.0,dummy_test_1to5_0.0_0.0_0.0_9.0_9.0,dummy_test_1to5_0.0_0.0_9.0_1.0_0.0,dummy_test_1to5_0.0_0.0_9.0_1.0_9.0,dummy_test_1to5_0.0_0.0_9.0_9.0_0.0,dummy_test_1to5_0.0_0.0_9.0_9.0_9.0,dummy_test_1to5_0.0_9.0_0.0_1.0_0.0,dummy_test_1to5_0.0_9.0_0.0_1.0_9.0,dummy_test_1to5_0.0_9.0_0.0_9.0_0.0,dummy_test_1to5_0.0_9.0_0.0_9.0_9.0,dummy_test_1to5_0.0_9.0_9.0_1.0_0.0,dummy_test_1to5_0.0_9.0_9.0_1.0_9.0,dummy_test_1to5_0.0_9.0_9.0_9.0_0.0,dummy_test_1to5_0.0_9.0_9.0_9.0_9.0,dummy_test_1to5_9.0_0.0_0.0_1.0_0.0,dummy_test_1to5_9.0_0.0_0.0_1.0_9.0,dummy_test_1to5_9.0_0.0_0.0_9.0_0.0,dummy_test_1to5_9.0_0.0_0.0_9.0_9.0,dummy_test_1to5_9.0_0.0_9.0_1.0_0.0,dummy_test_1to5_9.0_0.0_9.0_1.0_9.0,dummy_test_1to5_9.0_0.0_9.0_9.0_0.0,dummy_test_1to5_9.0_0.0_9.0_9.0_9.0,dummy_test_1to5_9.0_9.0_0.0_1.0_0.0,dummy_test_1to5_9.0_9.0_0.0_1.0_9.0,dummy_test_1to5_9.0_9.0_0.0_9.0_0.0,dummy_test_1to5_9.0_9.0_0.0_9.0_9.0,dummy_test_1to5_9.0_9.0_9.0_1.0_0.0,dummy_test_1to5_9.0_9.0_9.0_1.0_9.0,dummy_test_1to5_9.0_9.0_9.0_9.0_0.0,dummy_test_1to5_9.0_9.0_9.0_9.0_9.0,dummy_symptom_1to_5_0.0_0.0_0.0_0.0_0.0,dummy_symptom_1to_5_0.0_0.0_0.0_0.0_1.0,dummy_symptom_1to_5_0.0_0.0_0.0_1.0_0.0,dummy_symptom_1to_5_0.0_0.0_0.0_1.0_1.0,dummy_symptom_1to_5_0.0_0.0_1.0_0.0_0.0,dummy_symptom_1to_5_0.0_0.0_1.0_0.0_1.0,dummy_symptom_1to_5_0.0_0.0_1.0_1.0_0.0,dummy_symptom_1to_5_0.0_0.0_1.0_1.0_1.0,dummy_symptom_1to_5_0.0_1.0_0.0_0.0_0.0,dummy_symptom_1to_5_0.0_1

In [61]:
#X_train1 = h2o(X_train)
mynames = df_train.columns.to_list()
mynames = set(mynames)-set(['target'])
#mynames.remove()
y1 = df_train.target.values
X_train1, X_test1, y_train, y_test = train_test_split(df_train, y1, test_size=0.20, random_state=12345, stratify=y1)

X_train1 = h2o.H2OFrame(X_train1)
X_test1 = h2o.H2OFrame(X_test1)

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [62]:
# Identify predictors and response
x = list(mynames)
y = "target"

In [64]:
X_train1[y] = X_train1[y].asfactor()
X_test1[y] = X_test1[y].asfactor()

In [65]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml1 = H2OAutoML(max_models=20, seed=123, stopping_metric = 'misclassification')
aml1.train(x=x, y=y, training_frame=X_train1, validation_frame=X_test1)

AutoML progress: |
14:42:15.153: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
14:42:15.155: AutoML: XGBoost is not available; skipping it.

████████████████████████████████████████████████████████| 100%


In [66]:
lb = aml1.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_per_class_error,logloss,rmse,mse,auc,aucpr
StackedEnsemble_BestOfFamily_AutoML_20210808_144215,0.697466,1.29475,0.690946,0.477407,nan,nan
DeepLearning_grid__1_AutoML_20210808_144215_model_1,0.708346,1.50206,0.703996,0.49561,nan,nan
StackedEnsemble_AllModels_AutoML_20210808_144215,0.714571,1.29319,0.69129,0.477882,nan,nan
DeepLearning_grid__2_AutoML_20210808_144215_model_1,0.715609,1.3436,0.694443,0.482251,nan,nan
GBM_grid__1_AutoML_20210808_144215_model_5,0.724176,1.36919,0.718143,0.515729,nan,nan
DeepLearning_grid__3_AutoML_20210808_144215_model_1,0.724749,1.35032,0.707511,0.500572,nan,nan
DeepLearning_1_AutoML_20210808_144215,0.725598,1.35438,0.69628,0.484805,nan,nan
GBM_grid__1_AutoML_20210808_144215_model_1,0.731455,1.4195,0.716915,0.513967,nan,nan
GBM_grid__1_AutoML_20210808_144215_model_3,0.733745,1.43628,0.722225,0.521608,nan,nan
GBM_5_AutoML_20210808_144215,0.736786,1.38183,0.718339,0.516011,nan,nan


In [67]:
preds = aml1.leader.predict(X_test1)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [68]:
preds

predict,p0,p1,p2,p3,p4,p5,p6,p7,p8
3,0.00388713,0.134258,0.0379052,0.503793,2.73174e-05,0.0576619,0.00515758,0.257253,5.72464e-05
1,0.0616742,0.52565,0.314413,0.0818011,4.32147e-08,0.000282379,1.36492e-05,0.00586785,0.0102974
7,8.33119e-05,0.00635682,0.000541561,0.184957,0.000345892,0.282952,0.0461273,0.478633,2.62023e-06
3,0.00163802,0.101102,0.0124823,0.500232,2.43322e-05,0.0467442,0.00302761,0.334578,0.000172049
3,0.00775726,0.0816708,0.0211054,0.435716,8.56043e-05,0.0793689,0.00978671,0.364424,8.54625e-05
5,2.85861e-05,0.00199881,0.000179407,0.123366,0.00236932,0.400638,0.172623,0.298796,8.4244e-07
3,0.0025107,0.0737306,0.0169384,0.541782,4.48189e-05,0.0576913,0.00558712,0.301697,1.81755e-05
3,0.00193351,0.101371,0.0189446,0.546578,1.06907e-05,0.0372245,0.00228879,0.291606,4.30956e-05
3,0.0222405,0.298724,0.145121,0.445229,7.2454e-06,0.00960412,0.00106558,0.0772711,0.000737006
7,0.000957108,0.0339813,0.00853166,0.382775,5.23978e-05,0.152008,0.0199529,0.401724,1.80147e-05


In [69]:
#myscore.drop(columns = ['mytarget'], axis=1, inplace=True)
X_score1 = h2o.H2OFrame(X_score)
preds_h20 = aml1.leader.predict(X_score1)

Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


In [78]:
multi_class_df2=pd.DataFrame(preds_h20)

In [97]:
multi_class_df2 = pd.DataFrame()
multi_class_df2['sub_class_pred_h2o'] = h2o.as_list(preds_h20['predict']).predict.values
multi_class_df2['patient_id'] = df_valid.patient_id.values
multi_class_df2['h2o_sub_class'] = multi_class_df2['sub_class_pred_h2o'].apply(lambda x:class_decoding[x])
multi_class_df2.h2o_sub_class.value_counts()

Leigh syndrome                         3040
Cystic fibrosis                        2215
Mitochondrial myopathy                 1461
Diabetes                               1338
Tay-Sachs                               736
Leber's hereditary optic neuropathy     366
Alzheimer's                             162
Hemochromatosis                         144
Cancer                                    3
Name: h2o_sub_class, dtype: int64

In [98]:
multi_class_df2.to_csv('genetic_disorder_subclass_new_score_v3.csv',index=False)   # 36.35772